# Svchost DLL Search Order Hijack
IKEEXT and SessionEnv service, as they call LoadLibrary on files that do not exist within C:\Windows\System32\ by default. An attacker can place their malicious logic within the PROCESS_ATTACH block of their library and restart the aforementioned services "svchost.exe -k netsvcs" to gain code execution on a remote machine.

## Rule Content
```
- title: Svchost DLL Search Order Hijack
  id: 602a1f13-c640-4d73-b053-be9a2fa58b77
  status: experimental
  description: IKEEXT and SessionEnv service, as they call LoadLibrary on files that
    do not exist within C:\Windows\System32\ by default. An attacker can place their
    malicious logic within the PROCESS_ATTACH block of their library and restart the
    aforementioned services "svchost.exe -k netsvcs" to gain code execution on a remote
    machine.
  references:
  - https://posts.specterops.io/lateral-movement-scm-and-dll-hijacking-primer-d2f61e8ab992
  author: SBousseaden
  date: 2019/10/28
  tags:
  - attack.persistence
  - attack.defense_evasion
  - attack.t1073
  - attack.t1038
  - attack.t1112
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID: 7
      Image:
      - '*\svchost.exe'
      ImageLoaded:
      - '*\tsmsisrv.dll'
      - '*\tsvipsrv.dll'
      - '*\wlbsctrl.dll'
    filter:
      EventID: 7
      Image:
      - '*\svchost.exe'
      ImageLoaded:
      - C:\Windows\WinSxS\*
    condition: selection and not filter
  falsepositives:
  - Pentest
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((event_id:"7" AND process_path.keyword:(*\\svchost.exe) AND module_loaded.keyword:(*\\tsmsisrv.dll OR *\\tsvipsrv.dll OR *\\wlbsctrl.dll)) AND (NOT (event_id:"7" AND process_path.keyword:(*\\svchost.exe) AND module_loaded:("C\:\\Windows\\WinSxS\*"))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()